In [2]:
import pickle
import open3d as o3d
import numpy as np
from scipy.interpolate import interp1d
# Load the pickle file
with open('/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance/data/cloud_point_data4k.pkl', 'rb') as f:
    cloud_point_data = pickle.load(f)
    
print(f"Loaded {len(cloud_point_data)} point clouds")
cloud_point_data = cloud_point_data[2000:2003]
print(f"Loaded {len(cloud_point_data)} point clouds")

def filter_road(pcd):
    height_filter = pcd[:, 2] <= -2
    # height_filter_1 = pcd[:, 2] <= 2
    # valid_points = np.logical_and(height_filter)

    # Filter points and colors
    filtered_points = pcd[height_filter]
    return filtered_points

for frame_idx in range(len(cloud_point_data)):
    pcd = o3d.geometry.PointCloud()
    # road_points = filter_road(cloud_point_data[frame_idx])
    # pcd.points = o3d.utility.Vector3dVector(road_points)
    pcd.points = o3d.utility.Vector3dVector(cloud_point_data[frame_idx])
    
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=f"Point Cloud Frame {frame_idx+1}")
    view_ctl = vis.get_view_control()
    view_ctl.set_zoom(0.4)
    view_ctl.set_lookat([0.0, 0.0, 0.0])
    view_ctl.set_up([-1.0, 0.0, 0.0])

    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()

Loaded 4625 point clouds
Loaded 3 point clouds


In [16]:
# Find out the size of vehicle

import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN
from scipy.spatial import KDTree
from collections import defaultdict
from scipy.interpolate import interp1d
# Load the point cloud data for multiple frames
# cloud_point_data = np.load('/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/17-10-2024/cloud_point_data2.npy', allow_pickle=True)[300:310]
import numpy as np
import pickle
# Load the pickle file
with open('/Users/apple/Documents/UniversityDocument/Thesis/automative_self_driving_car/Autonomous-Navigation-and-Collision-Avoidance-Sytem-for-car/Collision_Avoidance/data/cloud_point_data4k.pkl', 'rb') as f:
    cloud_point_data = pickle.load(f)
    
print(f"Loaded {len(cloud_point_data)} point clouds")
cloud_point_data = cloud_point_data[1200:1202]
print(f"Loaded {len(cloud_point_data)} point clouds")

# Parameters
num_frames = len(cloud_point_data)

def filter_road(pcd):
    height_filter = pcd[:, 2] <= -1.98
    filtered_points = pcd[height_filter]
    circle_points = generate_3d_circle_points(radius=3.5, num_points=200)
    rectangle_points = generate_3d_rectangle_points(a=10, b=3, num_points=10000)
    
    combined_points = np.vstack((filtered_points, circle_points, rectangle_points))
    return combined_points

def filter_vehicle(pcd):
    height_filter = pcd[:, 2] <= 2
    filtered_points = pcd[height_filter]
    return filtered_points

def build_road_boundary_map(road_points):
    # Initialize the boundary maps
    x_boundary_map = defaultdict(lambda: {'min': float('inf'), 'max': -float('inf')})
    y_boundary_map = defaultdict(lambda: {'min': float('inf'), 'max': -float('inf')})
    
    # Populate the maps with min/max x and y values for each y and x, respectively
    for point in road_points:
        x, y, _ = np.round(point)  # We ignore the z value
        # x, y, _ = point # We ignore the z value
        
        # Update x map with the min/max y values
        x_boundary_map[x]['min'] = min(x_boundary_map[x]['min'], y)
        x_boundary_map[x]['max'] = max(x_boundary_map[x]['max'], y)
        
        # Update y map with the min/max x values
        y_boundary_map[y]['min'] = min(y_boundary_map[y]['min'], x)
        y_boundary_map[y]['max'] = max(y_boundary_map[y]['max'], x)
    
    return x_boundary_map, y_boundary_map

def is_point_on_road(x_boundary_map, y_boundary_map, point):
    x, y , z = point
    x, y, _ = np.round(point)
    if ((x_boundary_map[x]['min'] <= y <= x_boundary_map[x]['max']) and (y_boundary_map[y]['min'] <= x <= y_boundary_map[y]['max'])) and z > -1.98:
        return True

    return False

def generate_3d_circle_points(radius, num_points=1000):
    r = radius * np.sqrt(np.random.rand(num_points))
    theta = np.random.rand(num_points) * 2 * np.pi
    
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    z = np.full(num_points, -2)  # z = 0 for a circle in the xy-plane
    circle_points = np.vstack((x, y, z)).T
    return circle_points


def generate_3d_rectangle_points(a, b, num_points=1000, z_value=-2):

    # x = np.random.uniform(-8*a/10, 8*a/10, num_points)
    x = np.random.uniform(-2*a/10, 8*a/10, num_points)
    
    # Generate random y-values between -b/2 and b/2
    y = np.random.uniform(-b/2, b/2, num_points)
    
    # Set all z-values to the specified z_value
    z = np.full(num_points, z_value)

    # Combine x, y, and z into a single array of 3D points
    rectangle_points = np.vstack((x, y, z)).T

    return rectangle_points


# Example usage:
for frame_idx in range(len(cloud_point_data)):
    current_frame = cloud_point_data[frame_idx]
    current_frame = filter_vehicle(current_frame)
    # Filter road points
    road_points = filter_road(current_frame)
    
    x_boundary_map, y_boundary_map = build_road_boundary_map(road_points)

    road_pcd = o3d.geometry.PointCloud()
    road_pcd.points = o3d.utility.Vector3dVector(road_points)
    road_colors = np.full((road_points.shape[0], 3), [1, 0, 1])  # Green color for road points
    road_pcd.colors = o3d.utility.Vector3dVector(road_colors)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(current_frame)

    # Initialize an array to store colors (same size as the number of points)
    colors = np.zeros((current_frame.shape[0], 3))  # Default all to black (or any other default color)

    # Now, for each point in the current frame, check if it lies on the road
    for i, point in enumerate(current_frame):
        if is_point_on_road(x_boundary_map, y_boundary_map, point):
            # The point lies on the road, color it magenta (RGB: [1, 0, 1])
            colors[i] = [0, 1, 1]  # Magenta color
        else:
            # You can assign another color for points not on the road (optional)
            colors[i] = [0, 0, 1]  # Default to blue for points off the road
    # Assign the colors to the point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Visualize the point cloud with marked road points
    o3d.visualization.draw_geometries([pcd, road_pcd],
                                      zoom=0.4,
                                      front=[1.0, -1.0, 0.5],  # Set the side view of the camera
                                      lookat=[0.0, 0.0, 0.0],
                                      up=[0.0, 0.0, 1.0])


Loaded 4625 point clouds
Loaded 2 point clouds
[[80.0, -8.0, -2], [80.0, -1.0, -2]]
[[80.0, -8.0, -2], [80.0, -1.0, -2], [56.0, -2.0, -2], [56.0, -2.0, -2]]
[[80.0, -8.0, -2], [80.0, -1.0, -2], [56.0, -2.0, -2], [56.0, -2.0, -2], [55.0, -8.0, -2], [55.0, 5.0, -2]]
[[80.0, -8.0, -2], [80.0, -1.0, -2], [56.0, -2.0, -2], [56.0, -2.0, -2], [55.0, -8.0, -2], [55.0, 5.0, -2], [-53.0, -16.0, -2], [-53.0, -16.0, -2]]
[[80.0, -8.0, -2], [80.0, -1.0, -2], [56.0, -2.0, -2], [56.0, -2.0, -2], [55.0, -8.0, -2], [55.0, 5.0, -2], [-53.0, -16.0, -2], [-53.0, -16.0, -2], [-55.0, -8.0, -2], [-55.0, 10.0, -2]]
[[80.0, -8.0, -2], [80.0, -1.0, -2], [56.0, -2.0, -2], [56.0, -2.0, -2], [55.0, -8.0, -2], [55.0, 5.0, -2], [-53.0, -16.0, -2], [-53.0, -16.0, -2], [-55.0, -8.0, -2], [-55.0, 10.0, -2], [-56.0, -2.0, -2], [-56.0, 2.0, -2]]
[[80.0, -8.0, -2], [80.0, -1.0, -2], [56.0, -2.0, -2], [56.0, -2.0, -2], [55.0, -8.0, -2], [55.0, 5.0, -2], [-53.0, -16.0, -2], [-53.0, -16.0, -2], [-55.0, -8.0, -2], [-55.0, 10.